In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from CTL.causal_tree_learn import CausalTree
import numpy as np
import pickle

LoadError: ArgumentError: Package pandas not found in current path:
- Run `import Pkg; Pkg.add("pandas")` to install the pandas package.


In [31]:

print("### Load Data ######################")
data = pd.read_csv('data/preprocessed_app_data.csv')

### Load Data ######################


In [32]:
data

,spark.executor.cores,spark.executor.memory,spark.default.parallelism,spark.serializer,spark.task.cpus,executorDeserializeTime,executorDeserializeCpuTime,executorRunTime,resultSize,jvmGcTime,Execution_Time
0,7,6,29,0,2,7057.00,8.625728e+08,7521.75,840.0,1437.00,40424
1,7,6,37,0,1,7008.00,8.236984e+08,17763.75,840.0,2379.00,39742
2,6,6,16,1,2,6796.00,9.694377e+08,8766.35,918.0,1543.00,41370
3,6,7,31,0,2,6836.00,1.061869e+09,7971.10,840.0,1986.20,42098
4,5,1,10,1,2,6707.75,1.000639e+09,8739.15,918.0,1481.00,384288
...,...,...,...,...,...,...,...,...,...,...,...
1890,4,7,27,1,2,6353.30,7.977780e+08,6841.75,918.0,1207.05,46745
1891,2,11,49,0,2,6994.45,1.216635e+09,3986.25,840.0,852.35,66769
1892,5,9,40,1,2,6801.80,8.365599e+08,5834.70,918.0,986.35,50925
1893,5,6,15,1,1,7143.55,9.717473e+08,11350.55,918.0,2474.00,37600


In [33]:
y = data['Execution_Time'].values
treatment = data['spark.default.parallelism'].values
# # other configs
x = data.drop(['Execution_Time', 'spark.default.parallelism', 'executorDeserializeTime', 'executorDeserializeCpuTime', 'executorRunTime', 'resultSize', 'jvmGcTime'], axis=1).values
columns = data.drop(['Execution_Time', 'spark.default.parallelism', 'executorDeserializeTime', 'executorDeserializeCpuTime', 'executorRunTime', 'resultSize', 'jvmGcTime'], axis=1).columns

# other configs + other mediators
# x = data.drop(['Execution_Time', 'spark.default.parallelism', 'executorRunTime'], axis=1).values
# columns = data.drop(['Execution_Time', 'spark.default.parallelism', 'executorRunTime'], axis=1).columns


In [34]:
data

,spark.executor.cores,spark.executor.memory,spark.default.parallelism,spark.serializer,spark.task.cpus,executorDeserializeTime,executorDeserializeCpuTime,executorRunTime,resultSize,jvmGcTime,Execution_Time
0,7,6,29,0,2,7057.00,8.625728e+08,7521.75,840.0,1437.00,40424
1,7,6,37,0,1,7008.00,8.236984e+08,17763.75,840.0,2379.00,39742
2,6,6,16,1,2,6796.00,9.694377e+08,8766.35,918.0,1543.00,41370
3,6,7,31,0,2,6836.00,1.061869e+09,7971.10,840.0,1986.20,42098
4,5,1,10,1,2,6707.75,1.000639e+09,8739.15,918.0,1481.00,384288
...,...,...,...,...,...,...,...,...,...,...,...
1890,4,7,27,1,2,6353.30,7.977780e+08,6841.75,918.0,1207.05,46745
1891,2,11,49,0,2,6994.45,1.216635e+09,3986.25,840.0,852.35,66769
1892,5,9,40,1,2,6801.80,8.365599e+08,5834.70,918.0,986.35,50925
1893,5,6,15,1,1,7143.55,9.717473e+08,11350.55,918.0,2474.00,37600


In [35]:
x

array([[7, 6, 0, 2],
       [7, 6, 0, 1],
       [6, 6, 1, 2],
       ...,
       [5, 9, 1, 2],
       [5, 6, 1, 1],
       [6, 9, 1, 1]])

In [36]:
len(treatment)

1895

In [37]:
np.random.seed(0)

In [38]:
x_train, x_test, y_train, y_test, treat_train, treat_test = train_test_split(x, y, treatment, test_size=0.2, random_state=42)

In [39]:
len(x_train)

1516

In [40]:
# regular CTL
ctl = CausalTree(cont=True)
ctl.fit(x_train, y_train, treat_train)
ctl_predict = ctl.predict(x_test)

# honest CTL
cth = CausalTree(cont=True, honest=True)
cth.fit(x_train, y_train, treat_train)
cth_predict = cth.predict(x_test)

# val honest CTL
cthv = CausalTree(cont=True, val_honest=True)
cthv.fit(x_train, y_train, treat_train)
cthv_predict = cthv.predict(x_test)

# adaptive CT
ct_adaptive = CausalTree(weight=0.0, split_size=0.0, cont=True)
ct_adaptive.fit(x_train, y_train, treat_train)
ct_adaptive_predict = ct_adaptive.predict(x_test)

# honest CT
ct_honest = CausalTree(honest=True, weight=0.0, split_size=0.0, cont=True)
ct_honest.fit(x_train, y_train, treat_train)
ct_honest_predict = ct_honest.predict(x_test)


# if you want to plot a tree
# cthv.plot_tree(filename="output/all_configs_cont_trigger_tree")

ct_adaptive.plot_tree(features=columns, filename="output/parallelism_adaptive_ct_otherF", show_effect=True)
ct_honest.plot_tree(features=columns, filename="output/parallelism_honest_ct_otherF", show_effect=True)
ctl.plot_tree(features=columns, filename="output/parallelism_regular_ctl_otherF", show_effect=True)
cth.plot_tree(features=columns, filename="output/parallelism_honest_ctl_otherF", show_effect=True)
cthv.plot_tree(features=columns, filename="output/parallelism_valHonest_ctl_otherF", show_effect=True)

/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Other Configs + Mediators

In [ ]:
y = data['Execution_Time'].values
treatment = data['spark.default.parallelism'].values
# # other configs
x = data.drop(['Execution_Time', 'spark.default.parallelism', 'executorDeserializeTime', 'executorDeserializeCpuTime', 'executorRunTime', 'resultSize', 'jvmGcTime'], axis=1).values
columns = data.drop(['Execution_Time', 'spark.default.parallelism', 'executorDeserializeTime', 'executorDeserializeCpuTime', 'executorRunTime', 'resultSize', 'jvmGcTime'], axis=1).columns

# other configs + other mediators
# x = data.drop(['Execution_Time', 'spark.default.parallelism', 'executorRunTime'], axis=1).values
# columns = data.drop(['Execution_Time', 'spark.default.parallelism', 'executorRunTime'], axis=1).columns
